In [1]:
#-*- coding:utf-8 -*-
import urllib3
import json
import pickle

In [2]:
def etri(text): 
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU" 

    accessKey = "accessKey"
    analysisCode = "ner"
  
    requestJson = {
        "access_key": accessKey,
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }

    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )
    output = eval(str(response.data,"utf-8"))
    return output
    

In [3]:
text_list = []
per_list = []
org_list = []

with open('./full_text.txt', 'r') as f:
    while True:
        text = f.read(9500) # etri max input size: 10,000
        if not text:
            break
        output = etri(text)

        for o in output['return_object']['sentence']:
            text_list.append(o['text'])

            txt_output = open('./text_list5.txt', 'a')
            txt_output.write(f" \'{o['text']}\',")
            txt_output.close()

            nes = o['NE']
            tmp_per = []
            tmp_org = []
            for ne in nes:
                if ne['type'] == 'PS_NAME':
                    tmp_per.append(ne['text'])
                # elif 'OG' in ne['type']: # all organization
                elif ne['type'] == 'OGG_ECONOMY': # company name
                    tmp_org.append(ne['text'])
            per_list.append(tmp_per)
            org_list.append(tmp_org)


with open('text_list.pickle', 'wb') as f:
    pickle.dump(text_list, f, pickle.HIGHEST_PROTOCOL)
with open('per_list.pickle', 'wb') as f:
    pickle.dump(per_list, f, pickle.HIGHEST_PROTOCOL)
with open('org_list.pickle', 'wb') as f:
    pickle.dump(org_list, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# Save entities(person, organizaiton) predicted by ETRI api 
persons = [p.replace(" ", "") for p in persons]
persons = [p for p in persons if len(p)==3 or len(p)==2]
pd.Series(persons).to_csv('./etri_per_pred.csv', encoding='utf-8')

organizations = list(set([org for sublist in org_list for org in sublist]))
pd.Series(organizations).to_csv('./etri_org_pred.csv', encoding='utf-8')

In [16]:
# make pseudo-labeled training data by using predictions of ETRI API
import pandas as pd
answers = []
# for i in [39]:
for i in range(len(text_list)):
    text_splitted = text_list[i].split()
    org = org_list[i]
    per = per_list[i]
    answer = ['O' for _ in range(len(text_splitted))]
    for idx, t in enumerate(text_splitted):
        for o in org:
            if o in t:
                answer[idx] = 'ORG-B'
        for p in per:
            if p in t:
                answer[idx] = 'PER-B'
    _answer = ' '.join(answer)
    answers.append(_answer)

    
data_tuples = list(zip(text_list,answers))
data = pd.DataFrame(data_tuples, columns=['text','answer'])
data.to_csv('./naver_format_etri_1102.tsv', encoding='utf-8',index=False, header=False, sep ='\t')
data.head(4)

,text,answer
0,질석을 주원료로 하는 퇴비부숙제가 비료관리법상의 비료에 해당하는지 여부.,O O O O O O O O
1,신호등에 따라 교차로를 진행하는 차량 운전자에게 다른 차량이 신호를 위반하여 진행...,O O O O O O O O O O O O O O O O O O O O O O O ...
2,그리고 그 위임에 따른 구 간접투자법 시행령 제38조 제2항도 수익자총회의 의결을...,O O O O O O O O O O O O O O O O O O O O O O O
3,"행정청이 처분을 함에 있어 행정절차법 제23조 제1항, 제24조 제1항 소정의 이...",O O O O O O O O O O O O O O O O O O O O O O O ...
